In [1]:
import random
import varseek as vk
import os
# from tqdm import tqdm
# tqdm.pandas()

varseek_directory = os.path.dirname(os.path.abspath(""))

In [2]:
mutant_sequencing_depth = 256
wt_sequencing_depth = 0
condition = ['variant_type:equal=substitution']

number_of_variants_to_sample_per_condition = 10_000
read_length = 150
strand = None  # None for strand-agnostic (randomly-selected), "f" for forward, "r" for reverse, "both" for both - make sure this matches the reference genome (vk build command) - strand = True -> "f" or "r" here; strand = False -> None or "both" here - note that the strand is randomly selected per *transcript*, such that all drawn reads will come from the same strand no matter what
add_noise_sequencing_error=True
add_noise_base_quality=False
error_rate=0.0001  # only if add_noise_sequencing_error=True
error_distribution=(0.85, 0.1, 0.05)  # sub, del, ins  # only if add_noise_sequencing_error=True
max_errors=float("inf")  # only if add_noise_sequencing_error=True
seq_id_column="seq_ID"
var_column="mutation_cdna"
threads = 16

k = 51
w = 47

# Paths
vk_ref_out_dir = os.path.join(varseek_directory, "data", "vk_ref_out")
reference_out_dir = os.path.join(varseek_directory, "data", "reference")
sequences = os.path.join(reference_out_dir, "ensembl_grch37_release93", "Homo_sapiens.GRCh37.cdna.all.fa")
reference_genome_fasta = os.path.join(reference_out_dir, "ensembl_grch37_release93", "Homo_sapiens.GRCh37.dna.primary_assembly.fa")
out_dir_notebook = os.path.join(varseek_directory, "data", "synthetic_reads_test_easy")  #!!! change for each run
read_fq_path = os.path.join(out_dir_notebook, "synthetic_reads.fq")

mutation_metadata_df = os.path.join(vk_ref_out_dir, "variants_updated_exploded_filtered.csv")
read_df_out = f"{out_dir_notebook}/synthetic_reads.csv"
mutation_metadata_df_out = f"{out_dir_notebook}/mutation_metadata_df_updated_vk_info_exploded_with_synthetic_read_info.csv"

In [ ]:
if not os.path.exists(read_fq_path):
    simulated_df_dict = vk.sim(
        variants=mutation_metadata_df,
        number_of_variants_to_sample=number_of_variants_to_sample_per_condition,
        strand=strand,
        number_of_reads_per_variant_alt=mutant_sequencing_depth,
        number_of_reads_per_variant_ref=wt_sequencing_depth,
        k=k,
        w=w,
        read_length=read_length,
        seed=None,  # no need to pass in since it's set globally
        add_noise_sequencing_error=add_noise_sequencing_error,
        error_rate=error_rate,
        error_distribution=error_distribution,
        max_errors=max_errors,
        with_replacement=False,  # set to True if wanting to have some reads that might be identical
        sequences=sequences,
        seq_id_column=seq_id_column,
        var_column=var_column,
        var_id_column="header",
        variant_type_column="vcrs_variant_type",
        reference_out_dir=reference_out_dir,
        filters=condition,
        out=out_dir_notebook,
        reads_fastq_out=read_fq_path,
        reads_csv_out=read_df_out,
        variants_updated_csv_out=mutation_metadata_df_out,
    )

/home/jmrich/Desktop/varseek/varseek/varseek_sim.py:305: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  variants = pd.read_csv(variants)
12:15:06 - INFO - cannot find mutant sequence read parent
12:15:06 - INFO - running varseek build
12:15:06 - WARNING - If running a workflow with vk ref or kb ref, k should be an odd number between 1 and 63. Got k=147.
12:15:06 - INFO - Using COSMIC email from COSMIC_EMAIL environment variable: jmrich@caltech.edu
12:15:06 - INFO - Using COSMIC password from COSMIC_PASSWORD environment variable
12:15:54 - INFO - Using var_id_column 'header' as the variant header column.
12:20:56 - INFO - Removed 204 variant-containing reference sequences with length less than 150...
12:20:56 - WARNING - 
        5372870 variants correctly recorded (100.00%)
        204 variants removed (0.00%)
          0 variants missing seq_id or var_column (0.000%)
          0 entries removed due to having a duplicate entry 

**number_variants = 10_000**

number_variants x mutant_sequencing_depth = number_of_reads
10_000 x 256 = 2_560_000

number_of_reads + 1 (header row) = total_df_lines
2_560_000 + 1 = 2_560_001

number_of_reads x 4 (4 lines per read) = total_fastq_lines
2_560_000 x 4 = 10_240_000

In [4]:
!wc -l {read_df_out}
!wc -l {read_fq_path}

2560001 /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/synthetic_reads.csv
10240000 /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/synthetic_reads.fq


In [19]:
print("Haplotypecaller, same strand")
# !python scripts/run_gatk_haplotypecaller_for_benchmarking.py --synthetic_read_fastq /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/synthetic_reads.fq --skip_accuracy_analysis --out /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/haplotypecaller --reference_genome_fasta /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --reference_genome_gtf /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.87.gtf --star_genome_dir data/reference/ensembl_grch37_release93/star_reference --java /home/jmrich/opt/jdk-17.0.12+7/bin/java --picard_jar /home/jmrich/opt/picard.jar --threads 16 --genomes1000_vcf /data/reference/ensembl_grch37_release93/1000GENOMES-phase_3.vcf --disable_tool_default_read_filters --tmp_dir /data/tmp2

print(f"Number of reads in STAR BAM (out of {number_of_variants_to_sample_per_condition * mutant_sequencing_depth})")
!samtools view -c -F 4 /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/haplotypecaller/alignment/sample_Aligned.sortedByCoord.out.bam

print(f"Number of variants in unfiltered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/haplotypecaller/vcfs/haplotypecaller/haplotypecaller_output_unfiltered.g.vcf.gz | wc -l

print(f"Number of variants in filtered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/haplotypecaller/vcfs/haplotypecaller/haplotypecaller_output_filtered_applied.vcf.gz | wc -l

Haplotypecaller, same strand
Number of reads in STAR BAM (out of 2560000)
2590420
Number of variants in unfiltered VCF (out of 10000)
109
Number of variants in filtered VCF (out of 10000)
114


In [23]:
print("Mutect2, same strand")
# !python scripts/run_gatk_mutect2_for_benchmarking.py --synthetic_read_fastq /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/synthetic_reads.fq --skip_accuracy_analysis --out /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/mutect2 --reference_genome_fasta /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --reference_genome_gtf /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.87.gtf --star_genome_dir data/reference/ensembl_grch37_release93/star_reference --java /home/jmrich/opt/jdk-17.0.12+7/bin/java --picard_jar /home/jmrich/opt/picard.jar --threads 16 --genomes1000_vcf /data/reference/ensembl_grch37_release93/1000GENOMES-phase_3.vcf --disable_tool_default_read_filters --tmp_dir /data/tmp3

print(f"Number of reads in STAR BAM (out of {number_of_variants_to_sample_per_condition * mutant_sequencing_depth})")
!samtools view -c -F 4 /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/mutect2/alignment/sample_Aligned.sortedByCoord.out.bam

print(f"Number of variants in unfiltered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/mutect2/vcfs/mutect2/mutect2_output_unfiltered.g.vcf.gz | wc -l

print(f"Number of variants in filtered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/mutect2/vcfs/mutect2/mutect2_output_filtered_applied.vcf.gz | wc -l

Mutect2, same strand
Number of reads in STAR BAM (out of 2560000)
2590420
Number of variants in unfiltered VCF (out of 10000)
129
Number of variants in filtered VCF (out of 10000)
157


In [7]:
strand = "random"
read_fq_path_mixed_strands = os.path.join(out_dir_notebook, "synthetic_reads_mixed_strands.fq")
read_df_out = os.path.join(out_dir_notebook, "synthetic_reads_mixed_strands.csv")
mutation_metadata_df_out = os.path.join(out_dir_notebook, "mutation_metadata_df_updated_vk_info_exploded_with_synthetic_read_info_mixed_strands.csv")

if not os.path.exists(read_fq_path_mixed_strands):
    simulated_df_dict = vk.sim(
        variants=mutation_metadata_df,
        number_of_variants_to_sample=number_of_variants_to_sample_per_condition,
        strand=strand,
        number_of_reads_per_variant_alt=mutant_sequencing_depth,
        number_of_reads_per_variant_ref=wt_sequencing_depth,
        k=k,
        w=w,
        read_length=read_length,
        seed=None,  # no need to pass in since it's set globally
        add_noise_sequencing_error=add_noise_sequencing_error,
        error_rate=error_rate,
        error_distribution=error_distribution,
        max_errors=max_errors,
        with_replacement=False,  # set to True if wanting to have some reads that might be identical
        sequences=sequences,
        seq_id_column=seq_id_column,
        var_column=var_column,
        var_id_column="header",
        variant_type_column="vcrs_variant_type",
        reference_out_dir=reference_out_dir,
        filters=condition,
        out=out_dir_notebook,
        reads_fastq_out=read_fq_path_mixed_strands,
        reads_csv_out=read_df_out,
        variants_updated_csv_out=mutation_metadata_df_out,
    )

/home/jmrich/Desktop/varseek/varseek/varseek_sim.py:305: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  variants = pd.read_csv(variants)
13:36:44 - INFO - cannot find mutant sequence read parent
/home/jmrich/Desktop/varseek/varseek/varseek_sim.py:361: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  sim_data_df = pd.read_csv(update_df_out)
13:40:06 - INFO - Initial variant report
13:40:06 - INFO - Number of total variants: 5373074; VCRSs: 5373074; unique variants: 5373074; merged variants: 0

13:40:06 - INFO - variant_type equal substitution
13:40:10 - INFO - Number of total variants: 5143917 (229157 filtered); VCRSs: 5143917 (229157 filtered); unique variants: 5143917 (229157 filtered); merged variants: 0 (0 filtered)

13:40:10 - INFO - mutant_sequence_read_parent is_not_null None
13:40:16 - INFO - Number of total variants: 5143740 (177 filtered); VCRSs: 5143740 (177 fil

In [21]:
print("Haplotypecaller, mixed strands")
# !python scripts/run_gatk_haplotypecaller_for_benchmarking.py --synthetic_read_fastq /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/synthetic_reads_mixed_strands.fq --skip_accuracy_analysis --out /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/haplotypecaller --reference_genome_fasta /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --reference_genome_gtf /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.87.gtf --star_genome_dir data/reference/ensembl_grch37_release93/star_reference --java /home/jmrich/opt/jdk-17.0.12+7/bin/java --picard_jar /home/jmrich/opt/picard.jar --threads 16 --genomes1000_vcf /data/reference/ensembl_grch37_release93/1000GENOMES-phase_3.vcf --disable_tool_default_read_filters --tmp_dir /data/tmp2

print(f"Number of reads in STAR BAM (out of {number_of_variants_to_sample_per_condition * mutant_sequencing_depth})")
!samtools view -c -F 4 /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/haplotypecaller/alignment/sample_Aligned.sortedByCoord.out.bam

print(f"Number of variants in unfiltered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/haplotypecaller/vcfs/haplotypecaller/haplotypecaller_output_unfiltered.g.vcf.gz | wc -l

print(f"Number of variants in filtered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/haplotypecaller/vcfs/haplotypecaller/haplotypecaller_output_filtered_applied.vcf.gz | wc -l

Haplotypecaller, mixed strands
Number of reads in STAR BAM (out of 2560000)
2589084
Number of variants in unfiltered VCF (out of 10000)
9961
Number of variants in filtered VCF (out of 10000)
9936


In [26]:
print("Mutect2, mixed strands")
# !python scripts/run_gatk_mutect2_for_benchmarking.py --synthetic_read_fastq /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/synthetic_reads_mixed_strands.fq --skip_accuracy_analysis --out /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/mutect2 --reference_genome_fasta /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --reference_genome_gtf /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.87.gtf --star_genome_dir data/reference/ensembl_grch37_release93/star_reference --java /home/jmrich/opt/jdk-17.0.12+7/bin/java --picard_jar /home/jmrich/opt/picard.jar --threads 16 --genomes1000_vcf /data/reference/ensembl_grch37_release93/1000GENOMES-phase_3.vcf --disable_tool_default_read_filters --tmp_dir /data/tmp3

print(f"Number of reads in STAR BAM (out of {number_of_variants_to_sample_per_condition * mutant_sequencing_depth})")
!samtools view -c -F 4 /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/mutect2/alignment/sample_Aligned.sortedByCoord.out.bam

print(f"Number of variants in unfiltered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/mutect2/vcfs/mutect2/mutect2_output_unfiltered.g.vcf.gz | wc -l

print(f"Number of variants in filtered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/mutect2/vcfs/mutect2/mutect2_output_filtered_applied.vcf.gz | wc -l

Mutect2, mixed strands
Number of reads in STAR BAM (out of 2560000)
2589084
Number of variants in unfiltered VCF (out of 10000)
9986
Number of variants in filtered VCF (out of 10000)
7791
